In [ ]:
from ultralytics.models.yolo import YOLO

model_path = "/Users/vaibhavnakrani/yolo_dangerous_weapons/models/yolo/5_jan_2026_yolo11m/weights/best.pt"

model = YOLO(model_path)

model.to("mps")

# print(model.model)

In [ ]:
import torch
import matplotlib.pyplot as plt
import cv2
import numpy as np
from ultralytics.models.yolo import YOLO


def visualize_c2psa_attention(model, image_path, imgsz=640, save_path=None, alpha=0.65):
    feature_maps = {}
    
    def save_output_hook(name):
        def hook(module, input, output):
            feature_maps[name] = output.detach().cpu()
        return hook
    
    c2psa = model.model.model[10]
    hook_handle = c2psa.register_forward_hook(save_output_hook('c2psa_out'))
    
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    results = model(image_path, imgsz=imgsz)
    
    hook_handle.remove()
    
    print("\n" + "="*60)
    print("DETECTION RESULTS")
    print("="*60)
    
    boxes = results[0].boxes
    names = model.names
    
    if len(boxes) > 0:
        for i, box in enumerate(boxes):
            xyxy = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].cpu().numpy()
            cls_id = int(box.cls[0].cpu().numpy())
            cls_name = names[cls_id]
            
            print(f"\nDetection {i+1}:")
            print(f"  Class: {cls_name} (id={cls_id})")
            print(f"  Confidence: {conf:.3f}")
            print(f"  BBox [x1, y1, x2, y2]: [{xyxy[0]:.1f}, {xyxy[1]:.1f}, {xyxy[2]:.1f}, {xyxy[3]:.1f}]")
            print(f"  Width×Height: {xyxy[2]-xyxy[0]:.1f} × {xyxy[3]-xyxy[1]:.1f}")
    else:
        print("\nNo detections found.")
    
    print("="*60 + "\n")
    
    feat = feature_maps['c2psa_out']
    B, C, H, W = feat.shape
    
    activation = feat.abs().mean(dim=1).squeeze().numpy()
    p_low, p_high = np.percentile(activation, [2, 98])
    activation = np.clip(activation, p_low, p_high)
    activation = (activation - p_low) / (p_high - p_low + 1e-8)
    activation_resized = cv2.resize(activation, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_CUBIC)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(img_rgb)
    axes[0].set_title("Original Image")
    axes[0].axis("off")
    
    im = axes[1].imshow(activation_resized, cmap="turbo")
    axes[1].set_title(f"C2PSA Activation ({H}×{W} → upsampled)")
    axes[1].axis("off")
    plt.colorbar(im, ax=axes[1], fraction=0.046)
    
    axes[2].imshow(img_rgb)
    axes[2].imshow(activation_resized, cmap="turbo", alpha=alpha)
    
    for box in boxes:
        xyxy = box.xyxy[0].cpu().numpy()
        cls_id = int(box.cls[0].cpu().numpy())
        conf = box.conf[0].cpu().numpy()
        
        rect = plt.Rectangle(
            (xyxy[0], xyxy[1]), xyxy[2]-xyxy[0], xyxy[3]-xyxy[1],
            fill=False, edgecolor='lime', linewidth=2
        )
        axes[2].add_patch(rect)
        axes[2].text(
            xyxy[0], xyxy[1]-5, f"{names[cls_id]} {conf:.2f}",
            color='lime', fontsize=10, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.2', facecolor='black', alpha=0.7)
        )
    
    axes[2].set_title("Overlay with Detections")
    axes[2].axis("off")
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches="tight")
        print(f"Saved to: {save_path}")
    
    plt.show()
    
    return results, activation_resized

In [ ]:
model_path = "/Users/vaibhavnakrani/yolo_dangerous_weapons/models/yolo/5_jan_2026_yolo11m/weights/best.pt"
model = YOLO(model_path)
model.to("mps")

image_path = "/Users/vaibhavnakrani/Desktop/workspace/yolo_dataset_4_dec/images/test/voc_test_001371.jpg"

results, activation = visualize_c2psa_attention(
    model,
    image_path,
    imgsz=640,
    alpha=0.65
)

In [ ]:
import os
folder_path = "/Users/vaibhavnakrani/yolo_dangerous_weapons/notebooks/new-zealand"

image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

for image_file in image_files[:50]:
    image_path = os.path.join(folder_path, image_file)
    results, activation = visualize_c2psa_attention(
        model,
        image_path,
        imgsz=640,
        save_path=f"nz_attn/{image_file}",
        alpha=0.65
    )